In [31]:
#Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [105]:
#Load csv files 
HourlyMCP = pd.read_csv('../Data/ModelData/HourlyMCPDemo.csv', delimiter=',')
Fuel_Cost_Forecast_Normalized = pd.read_csv('../Data/ModelData/FuelCost_PriceDevelopment50years.csv', delimiter=',')
Generation_Asset_Data_New = pd.read_csv('../Data/ModelData/Generators_AssetData_New.csv', delimiter=',')
VRE_Generation_profile = pd.read_csv('../Data/ModelData/VRE_YearlyGenerationProfile_Normalized.csv', delimiter=',')

In [100]:
#show the first 5 rows of the data
#HourlyMCP.head()
#Fuel_Cost_Forecast_Normalized.head()
#Generation_Asset_Data_New.head()
VRE_Generation_profile.head()

,Season,Hour,WindZ1,WindZ2,PVZ1,PVZ2
0,1,1,0.500929,0.682578,0.0,0.0
1,1,2,0.449759,0.676634,0.0,0.0
2,1,3,0.461553,0.663912,0.0,0.0
3,1,4,0.461367,0.667905,0.0,0.0
4,1,5,0.410847,0.658432,0.0,0.0


In [73]:
#Define modelling time horizon
#Amount of years
Y = 1

#Amount of seasons in a year
S = 1

#Amount of hours in a season
H = 24

In [ ]:
#Initialize the model
m = gp.Model("InvestmentModel")

#Define the decision variables
#Capacity Investment in invetment opportunity i defined by Generation_Asset_Data_New['Unit #']
Inv_NewCap = m.addVars(Generation_Asset_Data_New['Unit #'], vtype=GRB.CONTINUOUS, name="Inv_NewCap")
#Energy generated by investment opportunity i in hour h
P_NewCap = m.addVars(Generation_Asset_Data_New['Unit #'], H, vtype=GRB.CONTINUOUS, name="P_NewCap")

#Extract the values from the pandas Series
HourlyMCP_values = HourlyMCP['MCP'].values
C_CapInv_values = Generation_Asset_Data_New['C_CapInv ($/MW)'].values
VarCost_values = Generation_Asset_Data_New['C_i ($/MWh)'].values

#Define the objective function
#Objective function is to maximize the profit
m.setObjective(30*365*gp.quicksum((P_NewCap[i,h] * (HourlyMCP_values[h] - VarCost_values[j])) for j, i in enumerate(Generation_Asset_Data_New['Unit #']) for h in range(H)) - gp.quicksum(Inv_NewCap[i] * C_CapInv_values[j] for j, i in enumerate(Generation_Asset_Data_New['Unit #'])), GRB.MAXIMIZE)

#Define the constraints
#Investment constraint
m.addConstrs((Inv_NewCap[i] >= 0 for i in Generation_Asset_Data_New['Unit #']), "InvestmentMin")
m.addConstrs((Inv_NewCap[i] <= Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'MaxInv (MW)'].values[0] for i in Generation_Asset_Data_New['Unit #']), "InvestmentMax")

#Budget constraint
m.addConstr(gp.quicksum(Inv_NewCap[i] * C_CapInv_values[j] for j, i in enumerate(Generation_Asset_Data_New['Unit #'])) <= 5000000000000000000, "Budget")

#Generation constraint
m.addConstrs((P_NewCap[i,h] <= Inv_NewCap[i] for i in Generation_Asset_Data_New['Unit #'] for h in range(H)), "GenerationMax")
m.addConstrs((P_NewCap[i,h] >= 0 for i in Generation_Asset_Data_New['Unit #'] for h in range(H)), "GenerationMin")

#VRE constraint 
#generation constraint for assets with technology=PV 
m.addConstrs(
    (
        P_NewCap[i, h] <= Inv_NewCap[i] * VRE_Generation_profile.loc[
            (VRE_Generation_profile['Hour'] == h+1) & (VRE_Generation_profile['Season'] == 1), 'PVZ1'
        ].values[0]
        for i in Generation_Asset_Data_New['Unit #']
        if Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'Technology'].values[0] == 'PV'
        for h in range(H)  # Skip the first hour
    ),
    "PVGenMax"
)
#generation constraint for assets with technology=Wind	
m.addConstrs(
    (
        P_NewCap[i, h] <= Inv_NewCap[i] * VRE_Generation_profile.loc[
            (VRE_Generation_profile['Hour'] == h+1) & (VRE_Generation_profile['Season'] == 1), 'WindZ1'
        ].values[0]
        for i in Generation_Asset_Data_New['Unit #']
        if Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'Technology'].values[0] == 'Wind'
        for h in range(H) 
    ),
    "WindGenMax"
)

#Solve the model
m.optimize()

# Check if the model found an optimal solution
if m.status == GRB.OPTIMAL:
    # Print the optimal solution
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    print('Obj:', m.objVal)
else:
    print("No optimal solution found. Status code:", m.status)

# Save the optimal solution to a df
# First df containts the investment decisions
df1 = pd.DataFrame(columns=['Unit #', 'Investment'])
df1['Unit #'] = Generation_Asset_Data_New['Unit #']
df1['Investment'] = [Inv_NewCap[i].x for i in Generation_Asset_Data_New['Unit #']]

# Second df contains the generation decisions
df2 = pd.DataFrame(columns=['Unit #', 'Hour', 'Generation'])
df2['Unit #'] = [i for i in Generation_Asset_Data_New['Unit #'] for h in range(H)]
df2['Hour'] = [h for i in Generation_Asset_Data_New['Unit #'] for h in range(H)]
df2['Generation'] = [P_NewCap[i,h].x for i in Generation_Asset_Data_New['Unit #'] for h in range(H)]

print(df1)
print(df2)






Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 969 rows, 400 columns and 1502 nonzeros
Model fingerprint: 0xa694a8a5
Coefficient statistics:
  Matrix range     [5e-02, 2e+06]
  Objective range  [2e+05, 3e+11]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+18]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 701 rows and 259 columns
Presolve time: 0.01s
Presolved: 268 rows, 141 columns, 536 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0894613e+10   1.016402e+03   0.000000e+00      0s
     146    1.9887434e+10   0.000000e+00   0.000000e+00      0s

Solved in 146 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.98

In [56]:
m.write("model.lp")